In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Env


In [ ]:
!pip install transformers==4.4.2
!pip install pytorch==1.7.0
!pip install sklearn
!pip install tqdm
!pip install numpy
!pip install scipy==1.2.1
!pip install jieba

     |████████████████████████████████| 2.0 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 12.5 MB/s 
     |████████████████████████████████| 3.3 MB 34.3 MB/s 
ERROR: Could not find a version that satisfies the requirement pytorch==1.7.0 (from versions: 0.1.2, 1.0.2)
ERROR: No matching distribution found for pytorch==1.7.0
     |████████████████████████████████| 24.8 MB 5.3 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!git clone https://github.com/yangjianxin1/GPT2-chitchat.git
!mv -f ./GPT2-chitchat/* ./

Cloning into 'GPT2-chitchat'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 96 (delta 15), reused 39 (delta 10), pack-reused 50
Unpacking objects: 100% (96/96), done.


In [ ]:
!git clone https://github.com/Chunshan-Theta/jieba_zh.git

Cloning into 'jieba_zh'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 43 (delta 7), reused 34 (delta 2), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [ ]:
! mkdir base_model

In [ ]:
# 下載wwm模型
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if __name__ == "__main__":
    file_id = "1DGSqKH7pDoj7N-j_vJZf_j_HQMnvWCAc"
    destination = './model/pytorch_model.bin'
    download_file_from_google_drive(file_id, destination)

    file_id = "1wuLDETNz-IViBKm4IfXvHvv-W3txrlP_"
    destination = './model/config.json'
    download_file_from_google_drive(file_id, destination)


    """
    1d-_D_wPVXV9IX-n9rkyyGdu77FWY9myx : 設定好之摘要模型
    1J_PV3m2OGseTbIyCfbejY28xaShHGevr : GPT 中文
    """
    file_id = "1d-_D_wPVXV9IX-n9rkyyGdu77FWY9myx"
    destination = './base_model/pytorch_model.bin'
    download_file_from_google_drive(file_id, destination)

    file_id = "1MXvBbuL_Kkxk977dAJRHNbDbVtOk6Pmz"
    destination = './base_model/config.json'
    download_file_from_google_drive(file_id, destination)

    file_id = "1wuyixzKs-Zi-uIpkwT2PxvfdOsVgnsXY"
    destination = './base_model/vocab.txt'
    download_file_from_google_drive(file_id, destination)

    


# train


## 準備資料

#### 準備資料方法1: 生成
透過詞性方式生成摘要

In [ ]:
# 下載wwm模型
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if __name__ == "__main__":
    file_id = "1O7LZz_YvO2KoMKhttUzE2unNVL6whiBx"
    destination = './data/source.txt'
    download_file_from_google_drive(file_id, destination)



In [ ]:
# import requests as rq

# def small_sent(sent):
#   res = rq.post("https://ckip.iis.sinica.edu.tw/api/corenlp/?ws=0",json={"text":sent})
#   print(res.text)
#   obj = res.json()['ws'][0]
#   small_sentence = ""
#   for unit in obj:
#     if unit['pos'].startswith("N") or unit['pos'].startswith("V"):
#       small_sentence+=unit['word']
#   return small_sentence

# small_sent("這片海非常清澈與乾淨")
import jieba_zh.posseg as pseg
import jieba_zh as jieba
class WordPos:
    """
    取出詞性
    """
    N_JIEBA = ["x","n", "ng", "nr", "nrfg", "nrt", "ns", "nt", "nz", "r", "rg", "rr", "rz"]
    V_JIEBA = ["v", "vd", "vg", "vi", "vn", "vt","post"]
    P_JIEBA = ["c","p","t"]
    A_JIEBA = ["a", "an", "ad", "ag","m","mg","mq","l"]

    @staticmethod
    def gen_word_type(word):
        words_types = []
        for w in word:
            flags = []
            subwords = pseg.cut(w)
            for subw in subwords:
                flags.append(subw.flag)

            words_types.append(flags)
        return words_types

    @staticmethod
    def cut_and_filter(sent):
      ws = [w for w in jieba.lcut(sent)]
      ws_tag = WordPos.gen_word_type(ws)
      re_ws = []
      print(ws,ws_tag)
      for w,tags in zip(ws, ws_tag):
        passtoken = False
        for t in tags:
          t = t.lower()
          if t in WordPos.V_JIEBA or t in WordPos.N_JIEBA or t in WordPos.A_JIEBA or t in WordPos.P_JIEBA:
            passtoken = True
        
        if passtoken:
          re_ws.append(w)
      
      return "".join(re_ws)
WordPos.cut_and_filter("這片海非常清澈與乾淨")

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.829 seconds.
Prefix dict has been built succesfully.


['這', '片海', '非常', '清澈', '與', '乾淨'] [['x'], ['q', 'n'], ['ADV'], ['Vi'], ['C'], ['Vi']]


'這片海清澈與乾淨'

In [ ]:
with open("./data/source.txt", "r") as f, open("./data/train.txt", "w") as o:
  idx = 0
  min_len = 15
  for line in f.readlines():
    question, ans = line.replace("\n","").split("\t")
    try:
      if len(question) >= min_len:
        
        question_small = WordPos.cut_and_filter(question)
        if len(question_small)<len(question):
          o.write(f"{question}\n{question_small}\n\n")
          idx+=1
          print(f"{question}\n{question_small}")

      if len(ans) >= min_len:
        ans_small = WordPos.cut_and_filter(ans)
        if len(ans_small)<len(ans):
          o.write(f"{ans}\n{ans_small}\n\n")
          idx +=1
          print(f"{ans}\n{ans_small}")
      
    except Exception as e:
      print(f"e:{e}")
    
    if idx >10000:
      break


串流輸出內容已截斷至最後 5000 行。
聯盟賺錢啊 傻瓜 有賭盤開4-0
['台電', '輸電', '建設', '沒', '弄好', '.', '你', '離', '岸風電', '就是', '屁', '...'] [['N'], ['Vi'], ['N'], ['ADV'], ['Vt'], ['x'], ['N'], ['P'], ['N'], ['C'], ['N'], ['m']]
台電輸電建設沒弄好.你離岸風電就是屁...
台電輸電建設弄好.你離岸風電就是屁...
['有沒有', '飆車族', '根', '黑道', '在', '北部', '的', '八卦'] [['ADV'], ['N'], ['M'], ['N'], ['P'], ['N'], ['T'], ['N']]
有沒有飆車族根黑道在北部的八卦
飆車族根黑道在北部的八卦
['精盡', '人亡', '！', '巴西', '16', '歲', '少年', '狂打', '42', '槍', '後', '暴斃'] [['i'], ['n', 'vg'], ['x'], ['N'], ['m'], ['M'], ['N'], ['Vt'], ['m'], ['N'], ['POST'], ['Vi']]
精盡人亡！巴西16歲少年狂打42槍後暴斃
人亡！巴西16歲少年狂打42槍後暴斃
['不', '說', '大家', '知道', '的', '中東', ' ', '東南亞', '現在', '也', '幾', '個', '月', '就', '炸一', '次', '了'] [['ADV'], ['Vt'], ['N'], ['Vt'], ['T'], ['N'], ['x'], ['N'], ['N'], ['ADV'], ['ADV'], ['M'], ['N'], ['ADV'], ['v', 'm'], ['M'], ['ASP']]
不說大家知道的中東 東南亞現在也幾個月就炸一次了
說大家知道的中東 東南亞現在個月炸一次
['加入', 'x', '權', ' ', '平權', ' ', '逢人', '就', '說', '你', '恐', 'x', ' ', '你', '74'] [['Vt'], ['x'], ['N'], ['x'], ['Vi'], ['x'], ['v', 'n'], ['ADV']

#### 準備資料方法2: 下載外部資料 

In [ ]:
# 下載wwm模型
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if __name__ == "__main__":

    file_id = "1s057dmtlkyAtlQ14YPbb8oL41M_mO-q1"
    destination = './data/source2.txt'
    download_file_from_google_drive(file_id, destination)

    


In [ ]:
import json
with open("./data/source2.txt", "r") as f, open("./data/train.txt", "w") as o:
  idx = 0
  min_len = 15
  for line in f.readlines():
    line = line.replace("\n","")
    obj = json.loads(line)
    print(obj)
    o.write(f"{obj['article']}\n{obj['summarization']}\n\n")
    idx +=1
    
    if idx >3000:
      break


{'summarization': '知情人透露章子怡怀孕后，父母很高兴。章母已开始悉心照料。据悉，预产期大概是12月底', 'article': '四海网讯，近日，有媒体报道称：章子怡真怀孕了!报道还援引知情人士消息称，“章子怡怀孕大概四五个月，预产期是年底前后，现在已经不接工作了。”这到底是怎么回事?消息是真是假?针对此消息，23日晚8时30分，华西都市报记者迅速联系上了与章子怡家里关系极好的知情人士，这位人士向华西都市报记者证实说：“子怡这次确实怀孕了。她已经36岁了，也该怀孕了。章子怡怀上汪峰的孩子后，子怡的父母亲十分高兴。子怡的母亲，已开始悉心照料女儿了。子怡的预产期大概是今年12月底。”当晚9时，华西都市报记者为了求证章子怡怀孕消息，又电话联系章子怡的亲哥哥章子男，但电话通了，一直没有人<Paragraph>接听。有关章子怡怀孕的新闻自从2013年9月份章子怡和汪峰恋情以来，就被传N遍了!不过，时间跨入2015年，事情却发生着微妙的变化。2015年3月21日，章子怡担任制片人的电影《从天儿降》开机，在开机发布会上几张合影，让网友又燃起了好奇心：“章子怡真的怀孕了吗?”但后据证实，章子怡的“大肚照”只是影片宣传的噱头。过了四个月的7月22日，《太平轮》新一轮宣传，章子怡又被发现状态不佳，不时深呼吸，不自觉想捂住肚子，又觉得不妥。然后在8月的一天，章子怡和朋友吃饭，在酒店门口被风行工作室拍到了，疑似有孕在身!今年7月11日，汪峰本来在上海要举行演唱会，后来因为台风“灿鸿”取消了。而消息人士称，汪峰原来打算在演唱会上当着章子怡的面宣布重大消息，而且章子怡已经赴上海准备参加演唱会了，怎知遇到台风，只好延期，相信9月26日的演唱会应该还会有惊喜大白天下吧。'}
{'summarization': '青海首次野外发现濒危大火烈鸟 尚不清楚具体来源', 'article': '中新社西宁11月22日电<Paragraph>(赵凛松)青海省林业厅野生动植物和自然保护区管理局高级工程师张毓22日向中新社记者确认：“经过中国林业科学院、中科院新疆生态与地理研究所和青海省林业厅的共同认定，出现在青海省海西州境内的三只体型较大的鸟为世界极度濒危的红鹳目红鹳科红鹳属的大红鹳。”11月18日，青海省海西州可鲁克湖—托素湖国家级陆生野生动物疫源疫病监测站在野外监测巡护过程中，在可鲁克

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'summarization': '荆州醉酒女被两男轮奸，事后两次翻供称自愿；因系嫌犯找到女孩男友承诺付5万赔偿，男友力劝女孩翻供', 'article': '荆楚网-楚天都市报<Paragraph>2014-12-30<Paragraph>13:21显示图片醉酒女遭两男子凌辱两次翻供称自愿<Paragraph>因包庇被刑拘20岁的李萌(化名)是个单纯的姑娘，洪湖人。今年4月发生的一件让她意想不到的事情，让她从被害者变为违法者。8个月前，两名男子将她轮奸，痛不欲生的她割腕自杀未果。民警很快抓住了一名嫌疑人，让她得到一丝安慰。而4月底至11月，李萌却先后两次向司法机关供称是“自愿”发生关系。事情很蹊跷。洪湖市检察院与公安局通过侦查，本月初揭开了翻供幕后的真相。女子喝醉后被凌辱4月中旬，洪湖汊河镇男子万某从苏州打工回老家，老同学朱某接他到洪湖城区一酒吧喝酒。朱某打电话让认识不久的李萌前来陪酒。李萌来后，三人一起喝了不少酒。李萌喝醉后，让朱某给她开房休息。4月22日凌晨，朱某和万某将李萌带到宾馆开了房。见李萌倒头就睡，两人顿起色心，将其轮奸。迷迷糊糊中，李萌感觉不对，便拨打了男友刘科(化名)的电话。刘科赶到宾馆，发现女友遭性侵后，立即报警。李萌痛不欲生，她砸破玻璃杯，欲割腕自杀，幸亏被男友夺下。在男友的安慰下，她打消了轻生念头。经检查确认，李萌当晚遭到两名男子的性侵。朱某和万某成为嫌疑人，民警对他们展开抓捕。7个月内两次翻供4月底的一天，一名中年男子约见李萌及其男友，对方称是朱某的舅舅。“放过朱某一马，一定不会让你们吃亏的。”男子说，只要你说是自愿发生关系的，给1.5万元。李萌听后，十分气愤，坚称按“法律”办。“朱某父亲得了癌症，活不了多久，他如果被抓了，谁来给他父亲送终？”李萌信以为真，在男友劝说下，同意与朱某的舅舅一起到派出所说明情况。面对民警，朱某的舅舅自称是李萌叔叔，称李萌报的是假案，并未被朱某强奸。而李萌称与朱某是自愿发生关系的。民警顿感事情蹊跷，便对中年男子的身份进行核实，揭穿了朱某舅舅的身份和如意算盘。3人灰溜溜地离开了派出所。5月5日，逃到苏州的万某被抓获，他对强奸一事供认不讳，而朱某一直在逃。随着万某的落网，李萌心里终于有了一丝安慰。今年11月，洪湖市法院公开开庭审理这起轮奸案。然而，在开庭后，李萌向法官、检察官和民警及辩护律师呈送“情况说明”，称自己

#### 訓練資料處理
將資料集轉換成訓練集格式

In [ ]:
!python preprocess.py --train_path data/train.txt --save_path data/train.pkl

2021-12-09 06:25:44,743 - INFO - preprocessing data,data path:data/train.txt, save path:data/train.pkl
2021-12-09 06:25:44,745 - INFO - there are 281 dialogue in dataset
100% 281/281 [00:00<00:00, 2581.79it/s]
2021-12-09 06:25:44,866 - INFO - finish preprocessing data,the result is stored in data/train.pkl
2021-12-09 06:25:44,866 - INFO - mean of dialogue len:101.86476868327402,median of dialogue len:92.0,max len:312


## 進行訓練

In [ ]:
import argparse
import math
import time
import torch
import torch.nn.functional as F
import torch.optim as optim
import logging
from datetime import datetime
import os
from torch.utils.data import Dataset, DataLoader
from os.path import join, exists
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from torch.nn import DataParallel
import transformers
import pickle
import sys
from pytorchtools import EarlyStopping
from sklearn.model_selection import train_test_split
from data_parallel import BalancedDataParallel
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, GPT2Config
from transformers import BertTokenizerFast
import pandas as pd
import torch.nn.utils.rnn as rnn_utils
import numpy as np
from dataset import MyDataset


class args_config:
    def __init__(self):
        self.log_path = "./logs.log"
        self.device = "0"
        self.vocab_path = "./base_model/vocab.txt"
        self.train_path = "./data/train.pkl"
        self.model_config = "./base_model/config.json"
        self.topk = 8
        self.topp = 0
        self.save_model_path = "model_new"
        self.repetition_penalty = 1.0
        self.max_len = 500
        self.max_history_len = 0
        self.no_cuda = False
        self.batch_size = 4
        self.warmup_steps = 200
        self.pretrained_model = "base_model" #"model_new/epoch5"
        self.val_num = 10
        self.num_workers = 0
        self.patience = 0
        self.gradient_accumulation_steps = 4
        self.max_grad_norm =2.0
        self.epochs = 15
        self.lr = 2.6e-5
        self.eps = 1.0e-09
        self.ignore_index = -100
        self.log_step = 1
def set_args():
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--device', default='3', type=str, required=False, help='设置使用哪些显卡')
    # parser.add_argument('--no_cuda', action='store_true', help='不使用GPU进行训练')
    # parser.add_argument('--vocab_path', default='vocab/vocab.txt', type=str, required=False,
    #                     help='词表路径')
    # parser.add_argument('--model_config', default='config/config.json', type=str, required=False,
    #                     help='设置模型参数')
    # parser.add_argument('--train_path', default='data/train.pkl', type=str, required=False, help='训练集路径')
    # parser.add_argument('--max_len', default=150, type=int, required=False, help='训练时，输入数据的最大长度')

    # parser.add_argument('--log_path', default='data/train.log', type=str, required=False, help='训练日志存放位置')
    # parser.add_argument('--log', default=True, help="是否记录日志")
    # parser.add_argument('--ignore_index', default=-100, type=int, required=False, help='对于ignore_index的label token不计算梯度')
    # # parser.add_argument('--input_len', default=200, type=int, required=False, help='输入的长度')
    # parser.add_argument('--epochs', default=100, type=int, required=False, help='训练的最大轮次')
    # parser.add_argument('--batch_size', default=4, type=int, required=False, help='训练的batch size')
    # parser.add_argument('--gpu0_bsz', default=10, type=int, required=False, help='0号卡的batch size')
    # parser.add_argument('--lr', default=2.6e-5, type=float, required=False, help='学习率')
    # parser.add_argument('--eps', default=1.0e-09, type=float, required=False, help='衰减率')
    # parser.add_argument('--log_step', default=1, type=int, required=False, help='多少步汇报一次loss')
    # parser.add_argument('--gradient_accumulation_steps', default=4, type=int, required=False, help='梯度积累')
    # parser.add_argument('--max_grad_norm', default=2.0, type=float, required=False)
    # parser.add_argument('--save_model_path', default='model', type=str, required=False,
    #                     help='模型输出路径')
    # parser.add_argument('--pretrained_model', default='', type=str, required=False,
    #                     help='预训练的模型的路径')
    # # parser.add_argument('--seed', type=int, default=None, help='设置种子用于生成随机数，以使得训练的结果是确定的')
    # parser.add_argument('--num_workers', type=int, default=0, help="dataloader加载数据时使用的线程数量")
    # parser.add_argument('--patience', type=int, default=0, help="用于early stopping,设为0时,不进行early stopping.early stop得到的模型的生成效果不一定会更好。")
    # parser.add_argument('--warmup_steps', type=int, default=4000, help='warm up步数')
    # # parser.add_argument('--label_smoothing', default=True, action='store_true', help='是否进行标签平滑')
    # parser.add_argument('--val_num', type=int, default=8000, help='验证集大小')
    # args = parser.parse_args()
    # return args
    return args_config()


def create_logger(args):
    """
    将日志输出到日志文件和控制台
    """
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    formatter = logging.Formatter(
        '%(asctime)s - %(levelname)s - %(message)s')

    # 创建一个handler，用于写入日志文件
    file_handler = logging.FileHandler(
        filename=args.log_path)
    file_handler.setFormatter(formatter)
    file_handler.setLevel(logging.INFO)
    logger.addHandler(file_handler)

    # 创建一个handler，用于将日志输出到控制台
    console = logging.StreamHandler()
    console.setLevel(logging.DEBUG)
    console.setFormatter(formatter)
    logger.addHandler(console)

    return logger


def collate_fn(batch):
    input_ids = rnn_utils.pad_sequence(batch, batch_first=True, padding_value=0)
    labels = rnn_utils.pad_sequence(batch, batch_first=True, padding_value=-100)
    return input_ids, labels


# def padding_batch(data_list, pad_id):
#     """
#     使用pad_id将data_list的每条数据，填充至data_list中最长的长度
#     :param data_list:
#     :param pad_id:
#     :return:
#     """
#     # 统计data_list中的最大长度
#     max_len = 0
#     for data in data_list:
#         max_len = max_len if max_len > len(data) else len(data)
#
#     # 对数据进行padding
#     new_data_list = []
#     for data in data_list:
#         new_data = data + [pad_id] * (max_len - len(data))
#         new_data_list.append(new_data)
#     return new_data_list


def load_dataset(logger, args):
    """
    加载训练集和验证集
    """
    logger.info("loading training dataset and validating dataset")
    train_path = args.train_path

    with open(train_path, "rb") as f:
        input_list = pickle.load(f)

    # 划分训练集与验证集
    val_num = args.val_num
    input_list_train = input_list[val_num:]
    input_list_val = input_list[:val_num]
    # test
    # input_list_train = input_list_train[:24]
    # input_list_val = input_list_val[:24]

    train_dataset = MyDataset(input_list_train, args.max_len)
    val_dataset = MyDataset(input_list_val, args.max_len)

    return train_dataset, val_dataset


def train_epoch(model, train_dataloader, optimizer, scheduler, logger,
                epoch, args):
    model.train()
    device = args.device
    # pad_id = args.pad_id
    # sep_id = args.sep_id
    ignore_index = args.ignore_index
    epoch_start_time = datetime.now()
    total_loss = 0  # 记录下整个epoch的loss的总和

    # epoch_correct_num:每个epoch中,output预测正确的word的数量
    # epoch_total_num: 每个epoch中,output预测的word的总数量
    epoch_correct_num, epoch_total_num = 0, 0

    for batch_idx, (input_ids, labels) in enumerate(train_dataloader):
        # 捕获cuda out of memory exception
        try:
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            outputs = model.forward(input_ids, labels=labels)
            logits = outputs.logits
            loss = outputs.loss
            loss = loss.mean()

            # 统计该batch的预测token的正确数与总数
            batch_correct_num, batch_total_num = calculate_acc(logits, labels, ignore_index=ignore_index)
            # 统计该epoch的预测token的正确数与总数
            epoch_correct_num += batch_correct_num
            epoch_total_num += batch_total_num
            # 计算该batch的accuracy
            batch_acc = batch_correct_num / batch_total_num

            total_loss += loss.item()
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            # 进行一定step的梯度累计之后，更新参数
            if (batch_idx + 1) % args.gradient_accumulation_steps == 0:
                # 更新参数
                optimizer.step()
                # 更新学习率
                scheduler.step()
                # 清空梯度信息
                optimizer.zero_grad()

            if (batch_idx + 1) % args.log_step == 0:
                logger.info(
                    "batch {} of epoch {}, loss {}, batch_acc {}, lr {}".format(
                        batch_idx + 1, epoch + 1, loss.item() * args.gradient_accumulation_steps, batch_acc, scheduler.get_lr()))

            del input_ids, outputs

        except RuntimeError as exception:
            if "out of memory" in str(exception):
                logger.info("WARNING: ran out of memory")
                if hasattr(torch.cuda, 'empty_cache'):
                    torch.cuda.empty_cache()
            else:
                logger.info(str(exception))
                raise exception

    # 记录当前epoch的平均loss与accuracy
    epoch_mean_loss = total_loss / len(train_dataloader)
    epoch_mean_acc = epoch_correct_num / epoch_total_num
    logger.info(
        "epoch {}: loss {}, predict_acc {}".format(epoch + 1, epoch_mean_loss, epoch_mean_acc))

    # save model
    logger.info('saving model for epoch {}'.format(epoch + 1))
    model_path = join(args.save_model_path, 'epoch{}'.format(epoch + 1))
    if not os.path.exists(model_path):
        os.mkdir(model_path)
    model_to_save = model.module if hasattr(model, 'module') else model
    model_to_save.save_pretrained(model_path)
    logger.info('epoch {} finished'.format(epoch + 1))
    epoch_finish_time = datetime.now()
    logger.info('time for one epoch: {}'.format(epoch_finish_time - epoch_start_time))

    return epoch_mean_loss


def validate_epoch(model, validate_dataloader, logger, epoch, args):
    logger.info("start validating")
    model.eval()
    device = args.device
    # pad_id = args.pad_id
    # sep_id = args.sep_id
    ignore_index = args.ignore_index
    epoch_start_time = datetime.now()
    total_loss = 0
    # 捕获cuda out of memory exception
    try:
        with torch.no_grad():
            for batch_idx, (input_ids, labels) in enumerate(validate_dataloader):
                input_ids = input_ids.to(device)
                labels = labels.to(device)
                outputs = model.forward(input_ids, labels=labels)
                logits = outputs.logits
                loss = outputs.loss
                loss = loss.mean()

                total_loss += loss.item()
                del input_ids, outputs

            # 记录当前epoch的平均loss
            epoch_mean_loss = total_loss / len(validate_dataloader)
            logger.info(
                "validate epoch {}: loss {}".format(epoch+1, epoch_mean_loss))
            epoch_finish_time = datetime.now()
            logger.info('time for validating one epoch: {}'.format(epoch_finish_time - epoch_start_time))
            return epoch_mean_loss
    except RuntimeError as exception:
        if "out of memory" in str(exception):
            logger.info("WARNING: ran out of memory")
            if hasattr(torch.cuda, 'empty_cache'):
                torch.cuda.empty_cache()
        else:
            logger.info(str(exception))
            raise exception


def train(model, logger, train_dataset, validate_dataset, args):
    train_dataloader = DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn,
        drop_last=True
    )
    validate_dataloader = DataLoader(validate_dataset, batch_size=args.batch_size, shuffle=True,
                                     num_workers=args.num_workers, collate_fn=collate_fn, drop_last=True)
    early_stopping = EarlyStopping(args.patience, verbose=True, save_path=args.save_model_path)
    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.epochs
    optimizer = transformers.AdamW(model.parameters(), lr=args.lr, eps=args.eps)
    # scheduler = transformers.WarmupLinearSchedule(optimizer, warmup_steps=args.warmup_steps, t_total=t_total)
    scheduler = transformers.get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    logger.info('starting training')

    # 用于记录每个epoch训练和验证的loss
    train_losses, validate_losses = [], []
    # 记录验证集的最小loss
    best_val_loss = 10000
    # 开始训练
    for epoch in range(args.epochs):
        # ========== train ========== #
        train_loss = train_epoch(
            model=model, train_dataloader=train_dataloader,
            optimizer=optimizer, scheduler=scheduler,
            logger=logger, epoch=epoch, args=args)
        train_losses.append(train_loss)

        # ========== validate ========== #
        validate_loss = validate_epoch(
            model=model, validate_dataloader=validate_dataloader,
            logger=logger, epoch=epoch, args=args)
        validate_losses.append(validate_loss)

        # 保存当前困惑度最低的模型，困惑度低，模型的生成效果不一定会越好
        if validate_loss < best_val_loss:
            best_val_loss = validate_loss
            logger.info('saving current best model for epoch {}'.format(epoch + 1))
            model_path = join(args.save_model_path, 'min_ppl_model'.format(epoch + 1))
            if not os.path.exists(model_path):
                os.mkdir(model_path)
            model_to_save = model.module if hasattr(model, 'module') else model
            model_to_save.save_pretrained(model_path)

        #  如果patience=0,则不进行early stopping
        if args.patience == 0:
            continue
        early_stopping(validate_loss, model)
        if early_stopping.early_stop:
            logger.info("Early stopping")
            break
    logger.info('training finished')
    logger.info("train_losses:{}".format(train_losses))
    logger.info("validate_losses:{}".format(validate_losses))


def caculate_loss(logit, target, pad_idx, smoothing=True):
    if smoothing:
        logit = logit[..., :-1, :].contiguous().view(-1, logit.size(2))
        target = target[..., 1:].contiguous().view(-1)

        eps = 0.1
        n_class = logit.size(-1)

        one_hot = torch.zeros_like(logit).scatter(1, target.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(logit, dim=1)

        non_pad_mask = target.ne(pad_idx)
        loss = -(one_hot * log_prb).sum(dim=1)
        loss = loss.masked_select(non_pad_mask).mean()  # average later
    else:
        # loss = F.cross_entropy(predict_logit, target, ignore_index=pad_idx)
        logit = logit[..., :-1, :].contiguous().view(-1, logit.size(-1))
        labels = target[..., 1:].contiguous().view(-1)
        loss = F.cross_entropy(logit, labels, ignore_index=pad_idx)
    return loss


def calculate_acc(logit, labels, ignore_index=-100):
    logit = logit[..., :-1, :].contiguous().view(-1, logit.size(-1))
    labels = labels[..., 1:].contiguous().view(-1)

    _, logit = logit.max(dim=-1)  # 对于每条数据，返回最大的index
    # 进行非运算，返回一个tensor，若labels的第i个位置为pad_id，则置为0，否则为1
    non_pad_mask = labels.ne(ignore_index)
    n_correct = logit.eq(labels).masked_select(non_pad_mask).sum().item()
    n_word = non_pad_mask.sum().item()
    return n_correct, n_word


def main():
    # 初始化参数
    args = set_args()

    # 设置使用哪些显卡进行训练
    os.environ["CUDA_VISIBLE_DEVICES"] = args.device

    args.cuda = not args.no_cuda

    if args.batch_size < 2048 and args.warmup_steps <= 4000:
        print('[Warning] The warmup steps may be not enough.\n' \
              '(sz_b, warmup) = (2048, 4000) is the official setting.\n' \
              'Using smaller batch w/o longer warmup may cause ' \
              'the warmup stage ends with only little data trained.')

    # 创建日志对象
    logger = create_logger(args)
    # 当用户使用GPU,并且GPU可用时
    args.cuda = torch.cuda.is_available() and not args.no_cuda
    device = 'cuda:0' if args.cuda else 'cpu'
    args.device = device
    logger.info('using device:{}'.format(device))

    # 初始化tokenizer
    tokenizer = BertTokenizerFast(vocab_file=args.vocab_path, sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]")
    args.sep_id = tokenizer.sep_token_id
    args.pad_id = tokenizer.pad_token_id
    args.cls_id = tokenizer.cls_token_id

    # 创建模型的输出目录
    if not os.path.exists(args.save_model_path):
        os.mkdir(args.save_model_path)

    # 创建模型
    if args.pretrained_model:  # 加载预训练模型
        model = GPT2LMHeadModel.from_pretrained(args.pretrained_model)
    else:  # 初始化模型
        model_config = GPT2Config.from_json_file(args.model_config)
        model = GPT2LMHeadModel(config=model_config)
    model = model.to(device)
    logger.info('model config:\n{}'.format(model.config.to_json_string()))
    assert model.config.vocab_size == tokenizer.vocab_size

    # 并行训练模型
    if args.cuda and torch.cuda.device_count() > 1:
        model = DataParallel(model).cuda()
        # model = BalancedDataParallel(args.gpu0_bsz, model, dim=0).cuda()
        logger.info("use GPU {} to train".format(args.device))

    # 计算模型参数数量
    num_parameters = 0
    parameters = model.parameters()
    for parameter in parameters:
        num_parameters += parameter.numel()
    logger.info('number of model parameters: {}'.format(num_parameters))

    # 记录参数设置
    logger.info("args:{}".format(args))

    # 加载训练集和验证集
    # ========= Loading Dataset ========= #
    train_dataset, validate_dataset = load_dataset(logger, args)

    train(model, logger, train_dataset, validate_dataset, args)


if __name__ == '__main__':
    main()


2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0
2021-12-09 06:40:28,735 - INFO - using device:cuda:0


[Warning] The warmup steps may be not enough.
(sz_b, warmup) = (2048, 4000) is the official setting.
Using smaller batch w/o longer warmup may cause the warmup stage ends with only little data trained.


串流輸出內容已截斷至最後 5000 行。
2021-12-09 06:45:26,873 - INFO - batch 61 of epoch 11, loss 2.234910488128662, batch_acc 0.529113924050633, lr [2.2749999999999997e-05]
2021-12-09 06:45:26,873 - INFO - batch 61 of epoch 11, loss 2.234910488128662, batch_acc 0.529113924050633, lr [2.2749999999999997e-05]
2021-12-09 06:45:26,873 - INFO - batch 61 of epoch 11, loss 2.234910488128662, batch_acc 0.529113924050633, lr [2.2749999999999997e-05]
2021-12-09 06:45:26,873 - INFO - batch 61 of epoch 11, loss 2.234910488128662, batch_acc 0.529113924050633, lr [2.2749999999999997e-05]
2021-12-09 06:45:26,873 - INFO - batch 61 of epoch 11, loss 2.234910488128662, batch_acc 0.529113924050633, lr [2.2749999999999997e-05]
2021-12-09 06:45:26,873 - INFO - batch 61 of epoch 11, loss 2.234910488128662, batch_acc 0.529113924050633, lr [2.2749999999999997e-05]
2021-12-09 06:45:26,873 - INFO - batch 61 of epoch 11, loss 2.234910488128662, batch_acc 0.529113924050633, lr [2.2749999999999997e-05]
2021-12-09 06:45:26,873 - I

In [ ]:
# 將訓練成果上傳至google drive
# !cp ./model_new/epoch5/pytorch_model.bin /content/drive/MyDrive/模型留存/gpt_summary/
# !cp ./model_new/epoch5/config.json /content/drive/MyDrive/模型留存/gpt_summary/

## new model test

In [ ]:
!python interact.py  --model_path ./model_new/epoch5 --max_history_len 1 --vocab_path ./base_model/vocab.txt --log_path ./logs.log --max_len 75

2021-12-09 06:23:54,548 - INFO - using device:cuda
开始和chatbot聊天，输入CTRL + Z以退出
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:我們都是學生
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:這樣也能讓你不後悔
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chatbot:這樣的決定不是你能够做的
user:做好最壞打算後便不顧一切的決定下去並記錄當時
chatbot:做好最壞打算後便會不顧一切的決定
user:做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
chat

#Test

### quick start


In [ ]:
!python interact.py --no_cuda --model_path ./model_new/epoch5 --max_history_len 1 --vocab_path ./vocab/vocab.txt --log_path ./logs.log

2021-12-01 17:45:45,185 - INFO - using device:cpu
开始和chatbot聊天，输入CTRL + Z以退出
user:夫人您在苦惱什麼呢？講出來比悶在心裡好吧！雖然我們可能幫不上什麼忙。
chatbot:夫人您在苦惱什麼呢講出來比悶在心裡好吧
user:那就去開啊！那有什麼問題？
chatbot:那有什麼問題那有什麼問題那有
user:阿姨你好，雖然有些唐突，但我可以幫你解決你的苦惱。你就把我當⋯⋯當仙姑好了！
chatbot:阿姨你好雖然有些唐突但我可以幫你解決
user:容易被他人影響是不是也代表者你在乎別人給妳的建議,所以又去重新思考到底是不是自已想要的.如果妳的朋友知道妳會反覆斟酌她的建議,應該會覺得開心自已的建議受到重視.
chatbot:容易被他人影響是不是也代表
user:Traceback (most recent call last):
  File "interact.py", line 136, in main
    text = input("user:")
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "interact.py", line 182, in <module>
    main()
  File "interact.py", line 177, in main
    samples_file.close()
KeyboardInterrupt


### interface


In [ ]:
import argparse
import math
import time
import torch
import torch.nn.functional as F
import torch.optim as optim
import logging
from datetime import datetime
import os
from torch.utils.data import Dataset, DataLoader
from os.path import join, exists
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from torch.nn import DataParallel
import transformers
import pickle
import sys
from pytorchtools import EarlyStopping
from sklearn.model_selection import train_test_split
from data_parallel import BalancedDataParallel
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, GPT2Config
from transformers import BertTokenizerFast
import pandas as pd
import torch.nn.utils.rnn as rnn_utils
import numpy as np
from dataset import MyDataset


In [ ]:
class set_args:
    def __init__(self):
        self.log_path = "./logs.log"
        self.device = "0"
        self.vocab_path = "./vocab/vocab.txt"
        self.model_path = "./model_new/epoch5"
        self.temperature = 1
        self.topk = 8
        self.topp = 0
        self.save_samples_path = "sample/"
        self.repetition_penalty = 1.0
        self.max_len = 50
        self.max_history_len = 5
        self.no_cuda = False

def create_logger(args):
    """
    将日志输出到日志文件和控制台
    """
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    formatter = logging.Formatter(
        '%(asctime)s - %(levelname)s - %(message)s')

    # 创建一个handler，用于写入日志文件
    file_handler = logging.FileHandler(
        filename=args.log_path)
    file_handler.setFormatter(formatter)
    file_handler.setLevel(logging.INFO)
    logger.addHandler(file_handler)

    # 创建一个handler，用于将日志输出到控制台
    console = logging.StreamHandler()
    console.setLevel(logging.DEBUG)
    console.setFormatter(formatter)
    logger.addHandler(console)

    return logger


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocab size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        # torch.topk()返回最后一维最大的top_k个元素，返回值为二维(values,indices)
        # ...表示其他维度由计算机自行推断
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value  # 对于topk之外的其他元素的logits值设为负无穷

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)  # 对logits进行递减排序
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits
args = set_args()
logger = create_logger(args)
# 当用户使用GPU,并且GPU可用时
args.cuda = torch.cuda.is_available()# and not args.no_cuda
device = 'cuda' if args.cuda else 'cpu'
logger.info('using device:{}'.format(device))
os.environ["CUDA_VISIBLE_DEVICES"] = args.device
tokenizer = BertTokenizerFast(vocab_file=args.vocab_path, sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]")
# tokenizer = BertTokenizer(vocab_file=args.voca_path)
model = GPT2LMHeadModel.from_pretrained(args.model_path)
model = model.to(device)
model.eval()
def summary(text):
    for _ in range(0,5):
      # text = "你好"
      # if args.save_samples_path:
      #     samples_file.write("user:{}\n".format(text))
      text_ids = tokenizer.encode(text, add_special_tokens=False)
      # history.append(text_ids)
      input_ids = [tokenizer.cls_token_id]  # 每个input以[CLS]为开头
      input_ids.extend(text_ids)
      input_ids.append(tokenizer.sep_token_id)
      # for history_id, history_utr in enumerate(history[-args.max_history_len:]):
      #     input_ids.extend(history_utr)
      #     input_ids.append(tokenizer.sep_token_id)
      input_ids = torch.tensor(input_ids).long().to(device)
      input_ids = input_ids.unsqueeze(0)
      response = []  # 根据context，生成的response
      # 最多生成max_len个token
      for _ in range(len(text_ids)):
          outputs = model(input_ids=input_ids)
          logits = outputs.logits
          next_token_logits = logits[0, -1, :]
          # 对于已生成的结果generated中的每个token添加一个重复惩罚项，降低其生成概率
          for id in set(response):
              next_token_logits[id] /= args.repetition_penalty
          next_token_logits = next_token_logits / args.temperature
          # 对于[UNK]的概率设为无穷小，也就是说模型的预测结果不可能是[UNK]这个token
          next_token_logits[tokenizer.convert_tokens_to_ids('[UNK]')] = -float('Inf')
          filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=args.topk, top_p=args.topp)
          # torch.multinomial表示从候选集合中无放回地进行抽取num_samples个元素，权重越高，抽到的几率越高，返回元素的下标
          next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
          if next_token == tokenizer.sep_token_id:  # 遇到[SEP]则表明response生成结束
              break
          response.append(next_token.item())
          input_ids = torch.cat((input_ids, next_token.unsqueeze(0)), dim=1)
          # his_text = tokenizer.convert_ids_to_tokens(curr_input_tensor.tolist())
          # print("his_text:{}".format(his_text))
      text = tokenizer.convert_ids_to_tokens(response)

      text = "".join(text)
      text = text.replace("#","")

      # print("chatbot:" + "".join(text))
      # if args.save_samples_path:
      #     samples_file.write("chatbot:{}\n".format("".join(text)))
      if len(text) > 0:
        break
    return text

print(summary("夫人您在苦惱什麼呢？講出來比悶在心裡好吧！雖然我們可能幫不上什麼忙。"))

2021-12-15 19:01:06,183 - INFO - using device:cuda


夫人您在苦惱什麼呢講出來比悶在心裡好吧雖然我們可能幫不上什麼忙


In [ ]:
file_index = 1
with open(f"./data/data.tsv","r") as f, open(f"./data/data_new.txt","w") as o:
  o.write(f"原文\t摘要\n")
  for idx, line in enumerate(f.readlines()):
    if idx <=0: 
      continue
    data = line.replace("\n","").split("\t")
    line = data[1][:700]
    print(idx)
    print("原文：", line)
    if len(line) > 5:
      gpt_sumy = summary(line)
    else:
      gpt_sumy = line
    print("摘要：",gpt_sumy)
    print("-----------------")
    o.write(f"{line}\t{gpt_sumy}\n")

串流輸出內容已截斷至最後 5000 行。
720
原文： 我沒事謝謝你，明明附近有更便宜的肉鋪，為什麼偏偏要挑那間相較之下比較貴的店呢？
摘要： 我沒事謝謝你明明附近有更便宜的肉鋪為什麼偏偏要挑那間
-----------------
721
原文： ㄎ
摘要： ㄎ
-----------------
722
原文： 夫人您在苦惱什麼呢？講出來比悶在心裡好吧！雖然我們可能幫不上什麼忙。
摘要： 但我們可以在家練習如何和孩子相處呢？
-----------------
723
原文： 那就去開啊！那有什麼問題？
摘要： 就去開啊！那有什麼問題就去
-----------------
724
原文： 妳在說什麼，我怎麼什麼都聽不懂？什麼江顯皎？什麼再生人？什麼時空旅人？現在2156年，我為什麼要回去100多年前的世界？
摘要： 聽不懂世界是什麼我怎麼樣什麼都不會懂
-----------------
725
原文： 琉星其實我有一件事一直想問妳。
摘要： 琉星其實我有一件事一直想問妳。
-----------------
726
原文： 你把這件事做好他們也會感到安慰
摘要： 你把這件事做好他們也會感到安慰
-----------------
727
原文： 作為一個局外人可能沒辦法幫你，頂多只能給你一些建議。
摘要： 作為一個局外人可能沒辦法幫你一些建議
-----------------
728
原文： 我沒事謝謝你，明明附近有更便宜的肉鋪，為什麼偏偏要挑那間相較之下比較貴的店呢？
摘要： 明明附近有更便宜的肉鋪為什麼偏偏要挑那個便宜的店呢
-----------------
729
原文： 支持自己的老公才正常吧
摘要： 支持自己的老公才正常
-----------------
730
原文： 夫人您在苦惱什麼呢？講出來比悶在心裡好吧！雖然我們可能幫不上什麼忙。
摘要： 我們可能幫不上忙別人也可能幫不上忙。但是我們可能可能只是想要讓自己生
-----------------
731
原文： 那就去開啊！那有什麼問題？
摘要： 就去開啊！那有什麼問題就去
-----------------
732
原文： 妳在說什麼，我怎麼什麼都聽不懂？什麼江顯皎？什麼再生人？什麼時空旅人？現在2156年，我為什麼要回去100多年前的世界？
摘要： 聽不懂什麼江顯皎是什麼都聽不懂


### 更新資料集

In [ ]:
from google.colab import files
files.download('./data/data_new.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
# for file_index in range(1,4):
#   with open(f"./data/source_1_{file_index}.txt","r") as f, open(f"./data/source_1_{file_index}_new.txt","w") as o:
#     o.write(f"原文\t摘要（DDparse模型）\t摘要（GPT模型）\n")
#     for idx, line in enumerate(f.readlines()):
#       if idx <=0: 
#         continue
#       line, sumy, _ = line.replace("\n","").split("\t")
#       print("原文：", line)
#       print("原先摘要：",sumy)
#       gpt_sumy = summary(line)
#       print("摘要（GPT模型）：",gpt_sumy)
#       print("-----------------")
#       o.write(f"{line}\t{sumy}\t{gpt_sumy}\n")
file_index = 1
with open(f"./data/source_1_{file_index}.txt","r") as f, open(f"./data/source_1_{file_index}_new.txt","w") as o:
  o.write(f"原文\t新摘要\t舊摘要\n")
  for idx, line in enumerate(f.readlines()):
    if idx <=0: 
      continue
    data = line.replace("\n","").split("\t")
    line, sumy = data[0], data[1]
    print(idx)
    print("原文：", line)
    print("舊摘要：",sumy)
    gpt_sumy = summary(line)
    print("新摘要：",gpt_sumy)
    print("-----------------")
    o.write(f"{line}\t{gpt_sumy}\t{sumy}\n")


1
原文： 有比較才能讓自己成為更傑出的人才啊，請將它當成有覺知的良性競爭並把出發點抓在爲了完成任務你好我也好的三贏狀態
舊摘要： 有比較才能讓自己成為人才，請將它當成為了完成任務你好我也好的良性競爭
新摘要： 不要把自己當做有覺知的良性竞争并把出發點抓在為了完成任務你好我也好的三贏狀態
-----------------
2
原文： 有時事事都打點計算好，生活中也會失去ㄧ些樂趣ㄉ
舊摘要： 事事都計算好會失去生活中的樂趣
新摘要： 有時事事都打點計算好，生活中也會失去一些樂趣ㄉ。
-----------------
3
原文： 做好最壞打算後便不顧一切的決定下去並記錄當時下決定的過程累積經驗後就不會再猶豫啦⋯⋯猶豫表深思熟慮多給自己一點時間嗯也無不好
舊摘要： 猶豫也是給自己一點時間
新摘要： 做好最壞打算後便不顧一切的決定
-----------------
4
原文： 做決定時不妨嘗試讓自己放手一搏吧幾次之後或許你會發現原來自己所做的決定並不是那麼的不好這時你就會想說那個很爽的過程了
舊摘要： 做決定幾次之後會發現原來自己做的決定並不是不好
新摘要： 做決定時也不妨嘗試讓自己放手一搏吧
-----------------
5
原文： 你已經覺得你已經是團隊裡的好幫手了就像那個小小的螺絲釘一樣或許不被看見但是卻是最重要的那一科
舊摘要： 你覺得你是團隊裡的好幫手了，不被看見卻是最重要的一顆
新摘要： 如果你是一位優秀的團隊的話你會很開心。
-----------------
6
原文： 相信自己是值得的你花了這麼多的時間來訓練自己為什麼不能夠爭取應得的報酬呢試試看吧
舊摘要： 相信自己是值得的
新摘要： 相信自己是值得的你花這麼多時間去訓練自己為什麼不能夠爭取應得的報酬
-----------------
7
原文： 總是托延是因為你想要更完美的結果，妳還在找更好的方式，先看見為何遲遲不動作的原因後妳就能快速前進了
舊摘要： 總是拖延是因為想要更完美的結果了
新摘要： 總是托延這是因為你想要更完美的結果後妳就能快速前進了
-----------------
8
原文： 雖然不知道自己想要什麼，但在摸索的過程中也讓自己有了許多歷練的不同事物的機會呀
舊摘要： 雖然不知道自己想要什麼，但摸索的過程多了歷練不同事物的機會
新摘要： 雖然不知道自己想要什麼，但在摸索的過

KeyboardInterrupt: ignored